In [37]:
import os, sys

# Mostrar el directorio actual y su contenido
print("Directorio actual:", os.getcwd())
print("Contenido de la carpeta:", os.listdir())

if "mlops_pipeline" in os.listdir():
    ruta_src = os.path.abspath("mlops_pipeline/src")
    sys.path.append(ruta_src)
    print("Ruta añadida al sistema:", ruta_src)
else:
    print("No se encontró la carpeta 'mlops_pipeline'. Verifique la estructura del proyecto.")

print("Rutas del sistema (últimas 3):")
for p in sys.path[-3:]:
    print(" ", p)


Directorio actual: /datasets/_deepnote_work
Contenido de la carpeta: ['mlops_pipeline', 'model_heuristic.pkl', 'preprocessor.pkl', 'telecom_churn.csv']
Ruta añadida al sistema: /datasets/_deepnote_work/mlops_pipeline/src
Rutas del sistema (últimas 3):
  /datasets/_deepnote_work/mlops_pipeline/src
  /datasets/_deepnote_work/mlops_pipeline/src
  /datasets/_deepnote_work/mlops_pipeline/src


In [40]:
import pandas as pd
import joblib
from ft_engineering import create_feature_pipeline, split_dataset
from heuristic_model import run_heuristic_model


In [16]:
# Entrenar nuevamente el modelo 
model, preprocessor = run_heuristic_model("telecom_churn.csv")

Modelo base (Logistic Regression)
Accuracy: 0.8546

Matriz de confusión:
[[552  18]
 [ 79  18]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.87      0.97      0.92       570
           1       0.50      0.19      0.27        97

    accuracy                           0.85       667
   macro avg       0.69      0.58      0.59       667
weighted avg       0.82      0.85      0.82       667



In [19]:
# Guardar el modelo y el preprocesador para “producción”
joblib.dump(model, "model_heuristic.pkl")
joblib.dump(preprocessor, "preprocessor.pkl")

print("Modelo y preprocesador guardados correctamente.")

Modelo y preprocesador guardados correctamente.


In [22]:
# Cargar el modelo guardado simulando la producción
model_loaded = joblib.load("model_heuristic.pkl")
preprocessor_loaded = joblib.load("preprocessor.pkl")

print("Modelo y preprocesador cargados para simulación.")

Modelo y preprocesador cargados para simulación.


In [25]:
# Crear un registro de prueba simulando un nuevo cliente
nuevo_cliente = pd.DataFrame({
    "AccountWeeks": [120],
    "DataUsage": [3.5],
    "CustServCalls": [1],
    "DayMins": [240],
    "DayCalls": [110],
    "MonthlyCharge": [70],
    "OverageFee": [10],
    "RoamMins": [15],
    "ContractRenewal": [1],
    "DataPlan": [1]
})


In [28]:
# Aplicar el pipeline al nuevo cliente
nuevo_transformado = preprocessor_loaded.transform(nuevo_cliente)

In [43]:
# Realizar la predicción
prediccion = model_loaded.predict(nuevo_transformado)
probabilidad = model_loaded.predict_proba(nuevo_transformado)[0][1]

print("Predicción del modelo (1 = Churn, 0 = No Churn):", int(prediccion[0]))
print(f"Probabilidad estimada de Churn: {probabilidad:.2f}")


Predicción del modelo (1 = Churn, 0 = No Churn): 0
Probabilidad estimada de Churn: 0.11


Insight: 

-Este modelo se probo con un nuevo cliente y se llego a la conclusión de que el modelo predijo que el cliente no hará churn, con una probabilidad del 11%, lo que indica un comportamiento estable.

In [4]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 18.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [7]:
# INTERFAZ STREAMLIT INTEGRADA

import streamlit as st
import pandas as pd
import joblib
import os

# Cargar modelo entrenado y preprocesador
MODEL_PATH = "model_heuristic.pkl"
PREPROCESSOR_PATH = "preprocessor.pkl"

if not (os.path.exists(MODEL_PATH) and os.path.exists(PREPROCESSOR_PATH)):
    st.error("No se encontraron los archivos del modelo entrenado.")
else:
    model = joblib.load(MODEL_PATH)
    preprocessor = joblib.load(PREPROCESSOR_PATH)
    st.success("Modelo y preprocesador cargados correctamente.")

# Configurar página Streamlit
st.set_page_config(page_title="Predicción de Deserción (Churn)", layout="centered")
st.title("Predicción de Deserción de Clientes (Churn)")
st.write("Sube un archivo CSV con los nuevos clientes para predecir si abandonarán el servicio.")

# Subir CSV
uploaded_file = st.file_uploader("Sube un archivo CSV con datos de clientes", type=["csv"])

if uploaded_file is not None:
    df_new = pd.read_csv(uploaded_file)
    st.subheader("Vista previa de los datos cargados:")
    st.dataframe(df_new.head())

    # Transformar y predecir
    X_new = preprocessor.transform(df_new)
    y_pred = model.predict(X_new)
    df_new["Predicción_Churn"] = y_pred

    # Mostrar resultados
    st.subheader("Resultados de la predicción:")
    st.dataframe(df_new)

    # Descargar resultados
    csv = df_new.to_csv(index=False).encode("utf-8")
    st.download_button("⬇Descargar resultados como CSV", csv, "predicciones_churn.csv", "text/csv")
else:
    st.info("Esperando archivo CSV para realizar predicciones.")

st.write("---")
st.caption("Desarrollado por Santiago López Gómez · Proyecto MLOps 2025 ")


2025-11-10 00:33:57.990 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 00:33:58.257 
  command:

    streamlit run /toolkit-cache/0.2.30/python3.10/kernel-libs/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-11-10 00:33:58.258 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 00:33:58.258 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 00:33:58.259 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 00:33:58.259 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 00:33:58.260 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 00:33

DeltaGenerator()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4f703592-4945-45ea-93be-011448982d02' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>